In [9]:
import cv2
import os
import tkinter as tk
from tkinter import filedialog

refPt = []
final_boundaries = []
image = []

def get_folder_name(initial_folder):
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Create a Toplevel window to act as a modal dialog
    dialog = tk.Toplevel(root)
    dialog.attributes("-topmost", True)  # Ensure it appears in front of every window
    dialog.grab_set()

    selected_folders = []
    while True:
        folder_path = filedialog.askdirectory(parent=dialog, title="Select a Folder", initialdir=initial_folder)

        if not folder_path:
            break  # Break the loop if the user cancels the selection

        selected_folders.append(folder_path)
    
    # Destroy the dialog window after the selection is made
    dialog.destroy()

    return selected_folders
    


def append_avi_extension(file_path):
    # Get the last part of the path
    base_name = os.path.basename(file_path)
    
    # Append ".avi" to the last part
    new_name = os.path.splitext(base_name)[0] + ".avi"

    return new_name

def click_and_crop(event, x, y, flags, param):
    global refPt, image, final_boundaries

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        final_boundaries.append((refPt[0], refPt[1]))
        cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("Select Rectangle", image)
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        clone = image.copy()
        cv2.rectangle(clone, refPt[0], (x, y), (0, 255, 0), 2)
        cv2.imshow("Select Rectangle", clone)

def images_to_video(image_folder, video_name, pt1, pt2, fps):
    """
    Convert a series of images to a video file.

    Parameters:
    - image_folder (str): Path to the folder containing the images.
    - video_name (str): Name of the output video file (e.g., 'output_video.mp4').
    - pt1 (tuple): Coordinates of the first point of the rectangle.
    - pt2 (tuple): Coordinates of the second point of the rectangle.
    - fps (float): Frames per second for the output video.

    Returns:
    - None
    """

    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]

    path_frame0 = os.path.join(image_folder, images[0])
    if pt1 is not None and pt2 is not None:
        frame0 = cv2.imread(path_frame0)[pt1[0]:pt2[0], pt1[1]:pt2[1], :]
    else:
        frame0 = cv2.imread(path_frame0)
    
    height, width, layers = frame0.shape

    video = cv2.VideoWriter(os.path.join(image_folder, video_name), cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

    for image in images:
        path_image = os.path.join(image_folder, image)
        if pt1 is not None and pt2 is not None:
            video.write(cv2.imread(path_image)[pt1[0]:pt2[0], pt1[1]:pt2[1], :])
        else:
            video.write(cv2.imread(path_image))

    cv2.destroyAllWindows()
    video.release()

def process_images(image_folder):
    global image
    video_name = append_avi_extension(image_folder)

    #Read in the image
    frame0 = cv2.imread(os.path.join(image_folder, "frame_0000.png"))
    fps = 10
    image = frame0.copy()

    #Fit the whole image to the screen
    max_display_height = 800
    if image.shape[0] > max_display_height:
        scale_factor = max_display_height / image.shape[0]
        image = cv2.resize(image, (int(image.shape[1] * scale_factor), max_display_height))
    else:
        scale_factor = 1

    #Create a window for the rectangle selection
    cv2.namedWindow("Select Rectangle")
    cv2.setMouseCallback("Select Rectangle", click_and_crop)
    cv2.imshow("Select Rectangle", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Print the coordinates of the selected rectangles
    if final_boundaries:
        print("Selected Rectangles:")
        for i, boundaries in enumerate(final_boundaries):
            print(f"Rectangle {i + 1}:", "Top-left:", boundaries[0], "Bottom-right:", boundaries[1])
        # Get back the coordinates in the right scale
        pt1 = int(boundaries[0][1]/scale_factor), int(boundaries[0][0]/scale_factor)
        pt2 = int(boundaries[1][1]/scale_factor), int(boundaries[1][0]/scale_factor)
    else:
        print("No rectangles selected.")
        pt1, pt2 = None, None

    # Write the cropped video to the desired folder
    images_to_video(image_folder, video_name, pt1, pt2, fps)
    

if __name__ == "__main__":
    #Get desired folder path and ouput video file name
    custom_home_folder = "C:\\Users\\lilap\\Studies\\Master ETHZ\\ARSL Semester Project\\Test videos" 
    selected_folders = get_folder_name(custom_home_folder)
    if selected_folders:
        print("Selected Folders:")
        for i, image_folder in enumerate(selected_folders):
            print(f"Folder {i + 1}: {image_folder}")
            process_images(image_folder)
    else:
        print("No folders selected.")



Selected Folders:
Folder 1: C:/Users/lilap/Studies/Master ETHZ/ARSL Semester Project/Test videos/14th Setup Bifurcation_2/0_5_mL_min_mini_piezo_1MHz_3Vpp
Selected Rectangles:
Rectangle 1: Top-left: (433, 109) Bottom-right: (1177, 677)
Folder 2: C:/Users/lilap/Studies/Master ETHZ/ARSL Semester Project/Test videos/14th Setup Bifurcation_2/0_5_mL_min_piezo_1MHz_3Vpp
Selected Rectangles:
Rectangle 1: Top-left: (433, 109) Bottom-right: (1177, 677)
Rectangle 2: Top-left: (430, 123) Bottom-right: (1176, 679)
Folder 3: C:/Users/lilap/Studies/Master ETHZ/ARSL Semester Project/Test videos/14th Setup Bifurcation_2/0_5_mL_min_piezo_1MHz_5Vpp
Selected Rectangles:
Rectangle 1: Top-left: (433, 109) Bottom-right: (1177, 677)
Rectangle 2: Top-left: (430, 123) Bottom-right: (1176, 679)
Rectangle 3: Top-left: (434, 119) Bottom-right: (1178, 679)
Folder 4: C:/Users/lilap/Studies/Master ETHZ/ARSL Semester Project/Test videos/14th Setup Bifurcation_2/1_5_mL_min_piezo_1MHz_10Vpp
Selected Rectangles:
Rectangl